# Modelos de lenguaje
Predecir el siguiente caractar en función de los caracteres anteriores
- Utilizar one-hot encoding para los caracteres
- Tamaño de ventana de 100
- Cantidad total de caracteres de la obra: 67561
- Cantidad de caracteres diferentes: 71

In [49]:
text = open('federico-garcia-lorca/bodas_de_sangre.txt').read()
number_of_chars = len(text)
all_chars = sorted(set(text))
print('-----------------------------------------------------------------------------------------------------')
print('-----------------------------------------------------------------------------------------------------')
print('Cantidad de caracteres: ' + str(number_of_chars))
print('Cantidad de caracteres unicos: ' + str(len(all_chars)))
print(all_chars)
print('-----------------------------------------------------------------------------------------------------')
print('-----------------------------------------------------------------------------------------------------')
print()
print(text[:997])

-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
Cantidad de caracteres: 67561
Cantidad de caracteres unicos: 71
['\n', ' ', '!', '(', ')', ',', '.', '1', '2', '3', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', '¿', 'É', 'Ñ', 'á', 'é', 'í', 'ñ', 'ó', 'ú']
-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------

BODAS DE SANGRE(1933)

Poema trágico en tres actos y siete cuadros

Personajes
 
Madre
Criada
Leonardo
Mozos
Novia
Vecina
Novio
Leñadores
Suegra
Muchachas
Padre de la nov

In [50]:
from RNN_utils import window_transform_text, encode_io_pairs

window_size = 100
step_size = 1
inputs, outputs = window_transform_text(text,window_size,step_size)

In [51]:
print(inputs[0])
print()
print('-----------------------------------------------------------------------------------------------------')
print('Salida:',outputs[0])
print('-----------------------------------------------------------------------------------------------------')
print()
print(inputs[1])
print('-----------------------------------------------------------------------------------------------------')
print('Salida:',outputs[1])
print('-----------------------------------------------------------------------------------------------------')

BODAS DE SANGRE(1933)

Poema trágico en tres actos y siete cuadros

Personajes
 
Madre
Criada
Leonar

-----------------------------------------------------------------------------------------------------
Salida: d
-----------------------------------------------------------------------------------------------------

ODAS DE SANGRE(1933)

Poema trágico en tres actos y siete cuadros

Personajes
 
Madre
Criada
Leonard
-----------------------------------------------------------------------------------------------------
Salida: o
-----------------------------------------------------------------------------------------------------


## ¿Como ingresamos a la red con caracteres?
### Categorical Data (One-hot encoding)

In [52]:
X, y = encode_io_pairs(text, all_chars, window_size,step_size)

In [53]:
print('Caracteres codificado')
print(X[0,0:5].astype(int))

Caracteres codificado
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [54]:
print('(N,T,D):')
print(X.shape)

(N,T,D):
(67461, 100, 71)


## RNN en Keras
Definamos una capa RNN

In [55]:
from keras.layers import SimpleRNN, Dense
from keras.models import Sequential

In [69]:
from keras.layers import TimeDistributed

### SimpleRNN

In [60]:
rnn_neurons = 2
time_steps = 1000# T
n_features = 71 # D
input_shape = (time_steps, n_features)
model = Sequential()
model.add(SimpleRNN(rnn_neurons, input_shape = input_shape))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_10 (SimpleRNN)    (None, 2)                 148       
Total params: 148
Trainable params: 148
Non-trainable params: 0
_________________________________________________________________


Observaciones:
- Crece la cantidad de parametros con la cantidad de time_steps?
- Como puedo utilizar un MLP para que tenga en cuenta los time_steps? Que desventajas tengo?
- rnn_neurons = 1, n_features = 1, por que es 3? Cuales son en el diagrama?

In [61]:
def get_number_of_parameters(rnn_neurons, n_features):
    parametros = rnn_neurons*n_features + rnn_neurons**2 + rnn_neurons
    return parametros

In [62]:
get_number_of_parameters(3,3)

21

### SimpleRNN + Dense para predecir caracteres
Armar modelo con los siguientes datos:
N,T,D = (67461, 100, 71)

In [74]:
# (67461, 100, 71)
rnn_neurons = 200 # Hyperparametro para jugar
time_steps = 100 # T
n_features =  71# D
input_shape = (time_steps, n_features)
model = Sequential()
model.add(SimpleRNN(rnn_neurons, input_shape = input_shape, return_sequences=True))
model.add(Dense(n_features, activation="softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_21 (SimpleRNN)    (None, 100, 200)          54400     
_________________________________________________________________
dense_17 (Dense)             (None, 100, 71)           14271     
Total params: 68,671
Trainable params: 68,671
Non-trainable params: 0
_________________________________________________________________


Observaciones:
- La cantidad de neuronas de la RNN queda para jugar (Overfitting, underfitting)